# Nerual Network From Scratch

I did problems 2-4 first and left this one for last. This caused me to run out of time. Thus, I followed a tutorial on creating this neural network from scratch because I wanted to be able to implement it and learn about it, but knew I would not be able to do it on my own in the amount of time I had. 

In [ ]:
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
#get mnist data, with one_hot encoding
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

C:\Users\ebish\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
x_train=mnist.train.images
y_train=mnist.train.labels
x_val=mnist.validation.images
y_val=mnist.validation.labels
x_test=mnist.test.images
y_test=mnist.test.labels

In [7]:
print(np.shape(x_train)[1])
print(np.shape(y_train)[1])

784
10


# Inputs, Outputs, Layers, Learning rate, cost function 

In [49]:
def lay_size(x,y):
    input_shape=np.shape(x_train)[1]
    num_nodes_hl=300
    output_shape=np.shape(y_train)[1]
    
    return (input_shape, num_nodes_hl,output_shape)


In [37]:
def initialize(input_shape, num_nodes_h1,output_shape):
    w1=np.random.randn(input_shape, num_nodes_h1)*.01
    w2=np.random.randn(num_nodes_h1,output_shape)*.01
    b1=np.zeros(shape=[num_nodes_h1])
    b2=np.zeros(shape=[output_shape])

    weights = {
        'w1': w1,
        'w2': w2   
    }

    biases = {
        'b1': b1,
        'b2': b2
    } 
    return weights, biases

# Implement forward propagation and compute the cost

In [68]:
def forward_prop(x,weights, biases):
    w1=weights['w1']
    w2=weights['w2']
    b1=biases['b1']
    b2=biases['b2']
    
    c1=np.dot(x,w1)+b1
    act1= c1*(c1>0)
    c2= (np.dot(act1,w2)+b2)
    act2= c2*(c2>0)
    
    new_values={
        'c1':c1,
        'act1': act1,
        'c2': c2,
        'act2': act2
    }
    
    return act2, new_values

In [69]:
# def softmax(x):
#     exps=np.exp(x)
#     ans=exps/np.sum(exps)
#     return ans

In [106]:
def cross_entropy(y_hat, y):
    p1=np.multiply(np.log(y_hat+.01),y)+np.multiply((1-y),np.log(1-y_hat+.01))
    c=-np.sum(p1)/np.shape(y)[0]
    c=np.squeeze(c)
    return c

In [107]:
# def cross_entropy(y_hat, y):
#     p=softmax(x)
#     m=np.shape(y)[0]
#     log_likelihood=-np.log(p[range(m),y.argmax(axis=1)])
#     loss=np.sum(log_likelihood)/m
#     return loss

# Compute the gradients of your cost function and back-propagation

In [108]:
# def gradient_cost_function(y_hat,y):
#     m=np.shape(y)[0]
#     grad=softmax(x)
#     grad[range(m),y.argmax(axis=1)]=grad[range(m),y.argmax(axis=1)]-1
#     grad=grad/m
#     return grad

In [128]:
def back_prop(x,y,weights, biases,new_values):
    m=np.shape(x)[0]
    
    w1=weights['w1']
    w2=weights['w2']
    
    act1=new_values['act1']
    act2=new_values['act2']
    
    dc2=act2-y
    dw2=(1/m)*np.dot(act1.T,dc2)
    db2=(1/m)*np.sum(dc2,axis=0,keepdims=True)
    dc1=np.multiply(np.dot(db2,w2.T),1-np.power(act1,2))
    dw1=(1/m)*np.dot(x.T,dc1)
    db1=(1/m)*np.sum(dc1,axis=0,keepdims=True)
    
    gradients={
        "dw1": dw1,
        "dw2": dw2,
        "db1": db1,
        "db2": db2
    }
    
    return gradients

In [129]:
def update(weights, biases, gradients, lr=.01):
    w1=weights['w1']
    w2=weights['w2']
    b1=biases['b1']
    b2=biases['b2']
    
    dw1=gradients['dw1']
    dw2=gradients['dw2']
    db1=gradients['db1']
    db2=gradients['db2']
    
    #update
    w1=w1-lr*dw1
    w2=w2-lr*dw2
    b1=b1-lr*db1
    b2=b2-lr*db2
    
    weights = {
    'w1': w1,
    'w2': w2   
    }

    biases = {
    'b1': b1,
    'b2': b2
    } 
    
    return weights, biases

In [130]:
def model(x,y,num_nodes_hl,its):
    
    input_shape=lay_size(x,y)[0]
    output_shape=lay_size(x,y)[2]
    
    weights, biases = initialize(input_shape, num_nodes_hl, output_shape)
    w1 = weights['w1']
    w2 = weights['w2']
    b1 = biases['b1']
    b2 = biases['b2']
    
    for i in range(0,its):
        act2,new_values=forward_prop(x,weights, biases)
        
        cost=cross_entropy(act2,y)
        
        gradients=back_prop(x,y,weights, biases,new_values)
        
        weights, biases=update(weights, biases, gradients, lr=.01)
        
        if i%100==0:
            print("cost after iteration %i: %f" %(i,cost))
        
    return weights, biases
        
        

In [131]:
def predict(weights, biases, x):
    act2,new_values=forward_prop(x,weights, biases)
    predictions=act2>.5
    
    return predictions

In [132]:
weights, biases=model(x_train,y_train,300,500)

cost after iteration 0: 4.309390
cost after iteration 100: 2.977251
cost after iteration 200: 2.884132
cost after iteration 300: 2.797523
cost after iteration 400: 2.720751


In [134]:
predictions = predict(weights, biases, x_train)
print ('Accuracy: %d' % float((np.dot(y_train,predictions.T) + np.dot(1-y_train,1-predictions.T))/float(y_train.size)*100) + '%')

MemoryError: 

In [138]:
predictions_test=predict(weights,biases,x_test)
print ('Accuracy: %d' % float((np.dot(y_test,predictions_test.T) + np.dot(1-y_test,1-predictions_test.T))/float(np.shape(y_test)[0])*100) + '%')

TypeError: only size-1 arrays can be converted to Python scalars

# Tensorflow verison 

In [144]:
import tensorflow as tf

In [152]:
#learning rate
lr = 0.01
#number of traning steps
num_steps =500
#number of batch_size
batch_size = 256

#network parameters
n_hidden_1 = 300
num_input = 784
num_classes = 10

In [153]:
tf.reset_default_graph()

In [154]:
#tf graph input
X = tf.placeholder(tf.float32,[None,num_input],name='X')
Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')

In [155]:
def nn_model(x):
    layer_one_out=tf.layers.dense(x,units=300,activation='relu')
    out=tf.layers.dense(layer_one_out,units=10)
    return out
    

In [163]:
#predicted labels
logits = nn_model(X)

#define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='loss')
#define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

#compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))

#compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')


In [164]:
#Initialize the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(num_steps):
        #fetch batch
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        #run optimization
        sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
        if i % 100 ==0:
            acc = sess.run(accuracy,feed_dict={X:batch_x, Y:batch_y})
            los=sess.run(loss,feed_dict={X:batch_x, Y:batch_y})
            print("step "+str(i)+",Training Accuracy= {:.3f}".format(acc))
            print('Cost: ',los)
    
    print("Training finished!")
    
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))

step 0,Training Accuracy= 0.430
Cost:  1.8955656
step 100,Training Accuracy= 0.953
Cost:  0.14390247
step 200,Training Accuracy= 0.969
Cost:  0.10751048
step 300,Training Accuracy= 0.988
Cost:  0.064897254
step 400,Training Accuracy= 0.984
Cost:  0.053828616
Training finished!
Testing Accuracy: 0.9697


# Insights

I ran out of time since this was the last thing I focused on out of all the questions so I did not finish everything for this one, but still found some valuable insight. 

The Neural Network from scratch took a lot longer to train. The tensorflow verison seemed to be 30 times faster. This is probably because they have coded tensorflow to be as efficient as possible where my code is to just get results. The cost is lower for the tensorflow, but not by as much as I would have thought.

The accuracy for for tensorflow on the testing accuracy is 97% which is pretty good. I couldn't get my accuracy for the numpy NN to work so I can't seem to compare that. Also, I didn't implement the Adam Optimier so I can't compare between those either. 